In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.simplefilter('ignore')

In [8]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util
from preprocessor import Processor

In [9]:
util = Util()
cleaner = DataCleaner()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# News

In [4]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
news_df=util.read_from_dvc("data/news.csv",repo,"news-v0",low_memory=False)

# cleaned_df=util.read_from_file('../data/news.csv')

In [5]:
news_df=news_df.sample(frac=1)

train_news=news_df.head(8)
test_news=news_df.tail(2)

In [6]:
full_processed_df=processor.prepare_text(train_news.copy())
description_processed=processor.prepare_text(train_news.copy(),columns=["Description"])[["Description","Analyst_Average_Score"]]
title_processed=processor.prepare_text(train_news.copy(),columns=["Title"])[["Title","Analyst_Average_Score"]]
body_processed=processor.prepare_text(train_news.copy(),columns=["Body"])[["Body","Analyst_Average_Score"]]
unprocessed=train_news

processed_test=processor.prepare_text(test_news)

In [8]:
full_processed_df['Description']=full_processed_df['Title'] + ", " + full_processed_df['Description'] + ", " + full_processed_df["Body"]
full_processed_df.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)

title_processed.rename(columns={'Title':'Description'},inplace=True)

body_processed.rename(columns={'Body':'Description'},inplace=True)

unprocessed['Description']=unprocessed['Title'] + ", " + unprocessed['Description'] + ", " + unprocessed["Body"]
unprocessed.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)

processed_test['Description']=processed_test['Title'] + ", " + processed_test['Description'] + ", " + processed_test["Body"]
processed_test.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)




In [11]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)


In [15]:
full_promp_trainer=full_promp_trainer.reset_index().drop(['index'],axis=1)

In [16]:
# full_promp_trainer=full_promp_trainer[['Domain','Title','Description','Body','timestamp','Analyst_Average_Score']]
full_promp_trainer.rename(columns={"Analyst_Average_Score":"Analyst Average Score"},inplace=True)
# full_promp_trainer=cleaner.convert_to_datetime(full_promp_trainer,['timestamp'])
# full_promp_trainer=full_promp_trainer.fillna('')
full_promp_trainer.head()

,Description,Analyst Average Score
0,"jet, thing, take, fund, johnson, taxpay, histo...",0.00
1,"collaps, scoan, nigeria, 7th, anniversari, sab...",0.00
2,"male, murder, arrest, femal, sap, 2021, 22, 13...",1.33
3,"woman, eskom, tweet, news24, dismiss, punish, ...",0.33
4,"construct, expect, repair, sector, loot, boost...",1.66


In [17]:

processor.prepare_tuner(full_promp_trainer)

tuner prepared successfuly


In [18]:
# uncomment these for first time use
# full_processed_df.to_csv("../data/trainer_news.csv",index=False)
# processed_test.to_csv("../data/test_news.csv",index=False)

# Job

In [10]:
job_df=pd.read_json("../data/job_description_train.json")
test_df=pd.read_json("../data/job_description_test.json")

In [12]:
job_df_train=job_df.copy()
job_df_test = test_df.copy()

processed_description=processor.prepare_job_description_text(job_df_train)
processed_test_description=processor.prepare_job_description_text(job_df_test)

display(processed_description.head())
display(processed_test_description.head())

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,DIPLOMA: Bachelor;DIPLOMA_MAJOR: Mechanical En...,"DEGREE_IN, EXPERIENCE_IN"
1,10+ years of software engineering work experie...,"EXPERIENCE: 10+ years, 5+ years;SKILLS: softwa...","EXPERIENCE_IN, DEGREE_IN"
2,3+ years Swift & Objective-C and experience wi...,EXPERIENCE: 3+ years;SKILLS: Swift & Objective-C;,EXPERIENCE_IN
3,8+ years experience in software engineering le...,"EXPERIENCE: 8+ years, 5+ years;SKILLS: softwar...",EXPERIENCE_IN
4,BS degree in Computer Science or related field...,DIPLOMA: BS;DIPLOMA_MAJOR: Computer Science;EX...,"DEGREE_IN, EXPERIENCE_IN"


,document,tokens,relations
0,"\nCurrently holding a faculty, industry, or go...",DIPLOMA: Ph.D.;DIPLOMA_MAJOR: machine learning...,"DEGREE_IN, EXPERIENCE_IN"
1,\n2+ years experience in the online advertisin...,EXPERIENCE: 2+ years;SKILLS: online advertisin...,"EXPERIENCE_IN, DEGREE_IN"
2,\nBA/BS\n5+ years of program or project manage...,"EXPERIENCE: 5+ years, 2+ years;SKILLS: project...",EXPERIENCE_IN
3,\nCurrently enrolled in a full-time degree pro...,DIPLOMA: Ph.D.;DIPLOMA_MAJOR: Computer Science;,DEGREE_IN
4,\nCurrently enrolled in a full-time degree pro...,DIPLOMA: Ph.D.;DIPLOMA_MAJOR: Computer Science;,DEGREE_IN


In [13]:
processed_test_description.shape

(11, 3)

In [14]:
processed_description.shape

(22, 3)

In [16]:
job_frames = [processed_description, processed_test_description.head(10)]

job_tuner_df = pd.concat(job_frames)

In [23]:
job_tuner_df=job_tuner_df.reset_index().drop(["index"],axis=1)

In [24]:
processor.prepare_job_description_tuner(job_tuner_df)

tuner prepared successfuly
